# cell

> Class representing a single code cell

In [ ]:
#| default_exp cell
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.misc_utils import print_red
from nbstata.code_utils import ending_sc_delimiter
from nbstata.stata_session import StataSession
from nbstata.magics import StataMagics
from fastcore.basics import patch_to

In [ ]:
#| hide
#| export
magic_handler = StataMagics()

In [ ]:
#| export
_final_delimiter_warning = (
    "Warning: Code cell (with #delimit; in effect) does not end in ';'. "
    "Exported .do script may behave differently from notebook. "
    "In v1.0, nbstata may trigger an error instead of just a warning."
)

In [ ]:
#| export
class Cell:
    """A class for managing execution of a single code cell"""                
    def _set_echo(self, echo_config):
        if echo_config == 'None':
            self.noecho = True
            self.echo = False
        elif echo_config == 'True':
            self.noecho = False
            self.echo = True
        else:
            self.noecho = False
            self.echo = False
    
    def __init__(self, kernel, code_w_magics, silent=False):
        self._set_echo(kernel.env['echo'])
        self.quietly = silent
        self.sc_delimiter = kernel.sc_delimiter
        self.stata_session = kernel.stata_session
        self.code = magic_handler.magic(code_w_magics, kernel, self)
       
    def run(self):
        if not self.code:
            return
        self.stata_session.dispatch_run(self.code, 
            quietly=self.quietly, echo=self.echo, sc_delimiter=self.sc_delimiter,
            noecho=self.noecho)
        self.sc_delimiter = self._check_ending_delimiter()

    def _check_ending_delimiter(self):
        _ending_sc_delimiter = ending_sc_delimiter(self.code, self.sc_delimiter)
        _final_character = self.code.strip()[-1]
        _code_missing_final_delimiter = (_ending_sc_delimiter
                                         and _final_character != ';')
        if _code_missing_final_delimiter:
            print_red(_final_delimiter_warning)
        return _ending_sc_delimiter

Some `Cell` functionality can be tested apart from a kernel:

In [ ]:
from nbstata.config import launch_stata
from fastcore.test import test_eq
from textwrap import dedent
from unittest.mock import Mock

In [ ]:
kernel1 = Mock()
kernel1.env = {'echo': 'None'}
kernel1.sc_delimiter = None

code_w_magics = '''disp "test output"'''
cell1 = Cell(kernel1, code_w_magics)
cell1.code

'disp "test output"'

In [ ]:
cell1.sc_delimiter = True
cell1._check_ending_delimiter()
cell1.sc_delimiter = False

In [ ]:
#| eval: false
launch_stata(splash=False)
kernel1.stata_session = StataSession()
cell1a = Cell(kernel1, code_w_magics)
cell1a.run()

test output


In [ ]:
code_w_magics = dedent('''\
    *%quietly
    disp "test output"
    ''')
cell2 = Cell(kernel1, code_w_magics)
test_eq(cell2.quietly, True)

In [ ]:
#| eval: false
cell2.run()

In [ ]:
kernel1.env = {'echo': 'True'}

code_w_magics = '''disp "test output"'''
cell3 = Cell(kernel1, code_w_magics)
test_eq(cell3.noecho, False)

In [ ]:
#| eval: false
cell3.run()

. disp "test output"
test output


In [ ]:
code_w_magics = dedent('''\
    *%noecho
    #delimit cr
    disp "test output"
    ''')
cell4 = Cell(kernel1, code_w_magics)
test_eq(cell4.noecho, True)

In [ ]:
#| eval: false
cell4.run()

test output


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()